In [1]:
import sys
import os
sys.path.append("address_score") 

#### Address Score Calculation

In [2]:
# Get the Greater Vancouver network map cache
from network_cache import cache_greater_vancouver_network

graph_path = "greater_vancouver.graphml"

if not os.path.exists(graph_path):
    print("Graph file not found. Generating it now...")
    cache_greater_vancouver_network(graph_path)
else:
    print("Graph file already exists. Skipping download.")

Graph file already exists. Skipping download.


In [3]:
# Calculating the road distances from desired addresses to houses
from compute_distances import compute_road_distances
# Desired addresses for scoring. This example uses the location of Northeastern University's Vancouver campus.
start_coords = (49.28069261104817, -123.11572527879807)
house_with_distances = compute_road_distances(start_coords)

📍 Loading Vancouver road network...
🚀 Running fast Dijkstra (single-source)...


In [4]:
# Calculating the address scores based on the distances
from get_address_score import get_address_score
house_with_address_scores = get_address_score(house_with_distances)
house_with_address_scores.head(10)

,Number,Address,List Date,Price,Days on market,Total floor area,Year Built,Age,Lot Size,Latitude,Longitude,Driving_Distance_m,Address_Scores
982,983,416-418 UNION STREET,9/9/2019,1600000,32,2301,1980,39,3150.0,49.277402,-123.095079,1660.572304,100.000000
961,962,521 UNION STREET,7/5/2019,1380000,56,2538,1901,118,3058.0,49.277693,-123.093001,1801.434931,98.506792
492,493,993 BROUGHTON STREET,4/3/2019,2600000,192,4026,1905,114,2882.0,49.286049,-123.133226,1833.572941,98.166115
1107,1108,518 KEEFER STREET,1/3/2020,1189000,47,3189,1910,110,3050.0,49.279169,-123.093047,1838.957340,98.109038
1006,1007,601 E PENDER STREET,1/11/2020,1670000,123,4100,1908,112,4150.0,49.280429,-123.091355,1891.673695,97.550221
500,501,643 E CORDOVA STREET,1/29/2020,1233000,21,2272,1901,119,3172.0,49.282316,-123.090425,2077.291595,95.582587
928,929,640 E GEORGIA STREET,5/27/2019,1860000,100,3335,1895,124,3050.0,49.278235,-123.090580,2184.427711,94.446896
923,924,703 HEATLEY AVENUE,7/25/2019,738000,196,1153,1901,119,1525.0,49.278251,-123.089910,2184.427711,94.446896
569,570,703 HEATLEY AVENUE,5/11/2020,750000,10,1153,1901,119,1525.0,49.278251,-123.089910,2184.427711,94.446896
698,699,763 UNION STREET,8/19/2019,1625000,7,1822,1905,114,3050.0,49.277710,-123.088270,2225.098618,94.015766


#### Safety Score Calculation